<a href="https://colab.research.google.com/github/Naomie25/DI-Bootcamp/blob/main/DailyChallenge_Week7_Day4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U pinecone==6.0.1
!pip install -U pinecone-notebooks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.4/421.4 kB 6.0 MB/s eta 0:00:00


In [2]:
import os
if not os.environ.get("PINECONE_API_KEY"):
    from pinecone_notebooks.colab import Authenticate
    Authenticate()

In [3]:
from pinecone import Pinecone
api_key = os.environ["PINECONE_API_KEY"]
environment = "us-west1-gcp"
pc = Pinecone(api_key=api_key, environment=environment)

In [4]:
query = "Tell me about Apple's products"

documents = [
    "Apple Inc. released the new iPhone with an improved camera and battery life.",
    "She ate a green apple after lunch because it was fresh and juicy.",
    "Apple's MacBook Pro is popular among designers and developers.",
    "The apple tree in our backyard is full of ripe fruit this season.",
    "Many people are excited about Apple’s upcoming augmented reality headset."
]


In [5]:
documents

['Apple Inc. released the new iPhone with an improved camera and battery life.',
 'She ate a green apple after lunch because it was fresh and juicy.',
 "Apple's MacBook Pro is popular among designers and developers.",
 'The apple tree in our backyard is full of ripe fruit this season.',
 'Many people are excited about Apple’s upcoming augmented reality headset.']

In [10]:
from pinecone import RerankModel
reranked_results = pc.inference.rerank(
   model="bge-reranker-v2-m3",
   query=query,
   documents=[{"id": str(i), "text": doc} for i, doc in enumerate(documents)],
   top_n=3
)



In [11]:
def show_reranked_results(query, matches):
    print(f"Query: {query}\n")
    print('Reranked Results:')
    for i, match in enumerate(matches):
        print(f'{str(i+1).rjust(4)}. Score: {match.score}')
        print(f'      Document: {match.document.text}')
        print('')
show_reranked_results(query,reranked_results.data)

Query: Tell me about Apple's products

Reranked Results:
   1. Score: 0.18892181
      Document: Apple's MacBook Pro is popular among designers and developers.

   2. Score: 0.038394015
      Document: Apple Inc. released the new iPhone with an improved camera and battery life.

   3. Score: 0.034488525
      Document: Many people are excited about Apple’s upcoming augmented reality headset.



In [12]:
pip install pandas torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [13]:
from pinecone import ServerlessSpec

# Le constructeur prend uniquement `cloud` et `region`
spec = ServerlessSpec(
    cloud="aws",        # ou "gcp", "azure"
    region="us-east-1"  # région supportée
)


In [14]:
from pinecone import Pinecone

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

index_name = "pinecone-reranker"

# Créer l'index serverless avec le spec
pc.create_index(
    name=index_name,
    dimension=384,               # ou 768, selon le modèle que tu utilises
    metric="cosine",             # "dotproduct" ou "euclidean" possibles aussi
    spec=spec
)


{
    "name": "pinecone-reranker",
    "metric": "cosine",
    "host": "pinecone-reranker-rawqzf3.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [15]:
import os, time, pandas as pd, torch
from pinecone import Pinecone, ServerlessSpec
import pandas as pd
import torch

cloud = os.getenv('PINECONE_CLOUD','aws')
region = os.getenv('PINECONE_REGION','us-east-1')
spec = ServerlessSpec(cloud=cloud,region=region)
index_name = "pinecone-reranker"

pc = Pinecone(api_key=api_key, environment=f"{cloud}-{region}")

In [17]:
if pc.has_index(index_name):
   pc.delete_index(name=index_name)
pc.create_index(
   name=index_name,
   dimension=384,
   # must match embedding vector size
  spec=spec
)


{
    "name": "pinecone-reranker",
    "metric": "cosine",
    "host": "pinecone-reranker-rawqzf3.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [18]:
import requests, tempfile

with tempfile.TemporaryDirectory() as tmpdir:
   file_path = os.path.join(tmpdir, "sample_notes_data.jsonl")
   url = "https://raw.githubusercontent.com/pinecone-io/examples/refs/heads/master/docs/data/sample_notes_data.jsonl"
   resp = requests.get(url)
   resp.raise_for_status()
   open(file_path, "wb").write(resp.content)
   df = pd.read_json(file_path, orient='records', lines=True)

In [19]:
print(df.head())

     id                                             values  \
0  P011  [-0.2027486265, 0.2769146562, -0.1509393603, 0...   
1  P001  [0.1842793673, 0.4459365904, -0.0770567134, 0....   
2  P002  [-0.2040648609, -0.1739618927, -0.2897160649, ...   
3  P003  [0.1889383644, 0.2924542725, -0.2335938066, -0...   
4  P004  [-0.12171068040000001, 0.1674752235, -0.231888...   

                                            metadata  
0  {'advice': 'rest, hydrate', 'symptoms': 'heada...  
1  {'tests': 'EKG, stress test', 'symptoms': 'che...  
2  {'HbA1c': '7.2', 'condition': 'diabetes', 'med...  
3  {'symptoms': 'cough, wheezing', 'diagnosis': '...  
4  {'referral': 'dermatology', 'condition': 'susp...  


In [20]:
index = pc.Index(index_name)
index.upsert_from_dataframe(df)

sending upsert requests:   0%|          | 0/100 [00:00<?, ?it/s]

{'upserted_count': 100}

In [21]:
def is_ready(idx):
   stats = idx.describe_index_stats()
   return stats.total_vector_count > 0

while not is_ready(index):
   time.sleep(5)
print(index.describe_index_stats())

{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 100}},
 'total_vector_count': 100,
 'vector_type': 'dense'}


In [22]:
from sentence_transformers import SentenceTransformer

def get_embedding(text):
   model = SentenceTransformer("all-MiniLM-L6-v2")  # e.g., "all-MiniLM-L6-v2"
   return model.encode(text)

In [26]:
question = "what if my patient has leg pain"
emb = get_embedding(question)

results = index.query(
    vector=emb.tolist() if hasattr(emb, "tolist") else emb,
    top_k=3,
    include_metadata=True
)
matches = sorted(results.matches, key=lambda m: m.score, reverse=True)



In [27]:
def show_results(q, matches):
    print(f"Question: {q}")
    for i, m in enumerate(matches):
        print(f"{i+1}. ID: {m.id}, Score: {m.score:.4f}, Metadata: {m.metadata}")

show_results(question, matches)


Question: what if my patient has leg pain
1. ID: P0100, Score: 0.5182, Metadata: {'advice': 'over-the-counter pain relief, stretching', 'symptoms': 'muscle pain'}
2. ID: P047, Score: 0.5011, Metadata: {'symptoms': 'back pain', 'treatment': 'physical therapy'}
3. ID: P095, Score: 0.5011, Metadata: {'symptoms': 'back pain', 'treatment': 'physical therapy'}


In [28]:
rerank_docs = [
    {
        "id": m.id,
        "reranking_field": "; ".join([f"{k}: {v}" for k, v in m.metadata.items()])
    }
    for m in matches
]

rerank_query = "What diagnostic steps should I take for a patient experiencing leg pain after recent surgery?"


In [29]:
reranked = pc.inference.rerank(
    model="bge-reranker-v2-m3",
    query=rerank_query,
    documents=rerank_docs,
    rank_fields=["reranking_field"],
    top_n=3
)
